In [ ]:
!nvidia-smi

Mon Sep 11 17:40:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install transformers
!pip3 install accelerate
!pip3 install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=b1db233b1fcef3c73a60a1a096cb2d6961c3db1daecf9e20db3b64438812bfbe
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from Data import CS10KDataset, MixDataset
import torch
import matplotlib.pyplot as plt
import re
from rouge_score import rouge_scorer
from tqdm import tqdm
import numpy as np
import pickle
import random

In [ ]:
DATASET = "CS10K"
#DATASET = "Mix"
DATA_DIR = 'drive/MyDrive/Multimodal-Image-Captioning/Data/'+DATASET+'/'
NUM_BATCHES = 400
RESIZE_SHAPE = None
VERBOSE = False
NUM_WORKERS = 10

if DATASET == "CS10K":
    dataset = CS10KDataset(DATA_DIR, NUM_BATCHES, RESIZE_SHAPE, VERBOSE, NUM_WORKERS)
else:
    dataset = MixDataset(DATA_DIR, NUM_BATCHES, RESIZE_SHAPE, VERBOSE, NUM_WORKERS)
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

In [ ]:
def avg(l):
    return sum(l) / len(l)


def loss_fn(target_ids, output):
    target = torch.nn.functional.one_hot(label_ids, num_classes=50272)[0]
    output = torch.nn.functional.softmax(output.logits[0],dim=-1)
    loss = -1*torch.sum(target*torch.log(output))
    return loss

def get_prefix(data_point,typ):
    if typ == 0:
        prefix = 'Question: What is the caption for this Figure?\nAnswer: '
    elif typ == 1:
        prefix = f'Question: What is the caption for this Figure with Title: "{data_point["title"]}"?\nAnswer: '
    else:
        prefix = f'Question: What is the caption fro this Figure with Title: "{data_point["title"]}" and Abstract: "{data_point["abstract"]}"?\nAanswer: '
    return prefix


def forward(model, processor, data_point, typ):
    figure = data_point['figure']
    if len(figure.shape) ==2:
        figure = np.dstack([figure, figure, figure])

    prefix = get_prefix(data_point,typ)
    label = prefix + data_point['label'] + "</s>"
    inputs = processor(figure, label, return_tensors="pt").to("cuda")
    label_ids = processor.tokenizer(label, return_tensors="pt").input_ids.to("cuda")

    output = model(**inputs, labels=label_ids)
    return output, output.loss

def generate(model, processor, data_point, typ):
    figure = data_point['figure']
    if len(figure.shape) ==2:
        figure = np.dstack([figure, figure, figure])
    prefix = get_prefix(data_point,typ)
    inputs = processor(figure, prefix, return_tensors="pt").to("cuda")

    output = processor.decode(model.generate(**inputs, temperature=0.95, max_new_tokens=50, do_sample=True, top_k=50)[0], skip_special_tokens=True)
    #output = processor.decode(model.generate(**inputs, max_new_tokens=50)[0], skip_special_tokens=True)
    return output

def validation(model, processor, dataset, scorer, typ, num_batches,per_batch):
    model.eval()
    with torch.no_grad():
        idxs = list(range(280, 340))
        random.shuffle(idxs)
        idxs = idxs[:num_batches]
        losses = []
        rouge1 = []
        rouge2 = []
        rougeL = []
        generated_texts = []
        for batch_idx, idx in enumerate(idxs):
            print(f'Validation Batch {batch_idx+1}/{num_batches}')
            batch = dataset[idx]
            random.shuffle(batch)
            batch = batch[:per_batch]
            loop = tqdm(batch)
            for data_point in loop:
                try:
                  output, loss = forward(model, processor, data_point, typ)
                  losses.append(loss.item())
                  generated = generate(model, processor, data_point, typ)
                  scores = scorer.score(data_point['label'].lower(), generated.lower())
                  rouge1.append(scores['rouge1'].fmeasure)
                  rouge2.append(scores['rouge2'].fmeasure)
                  rougeL.append(scores['rougeL'].fmeasure)
                  loop.set_description(f'Validation Loss: {losses[-1]} Rouge1: {rouge1[-1]} Rouge2: {rouge2[-1]} RougeL: {rougeL[-1]}')
                  generated_texts.append(generated)
                except:
                  continue

    model.train()
    return avg(losses), avg(rouge1), avg(rouge2), avg(rougeL), generated_texts

In [ ]:
  validation_loss, validation_rouge1_score, validation_rouge2_score, validation_rougeL_score, generated = validation(model, processor, dataset, scorer, TYPE, 4, 5)


Validation Batch 1/4


Validation Loss: 7.502792835235596 Rouge1: 0.15384615384615383 Rouge2: 0.0 RougeL: 0.15384615384615383: 100%|██████████| 5/5 [00:01<00:00,  2.82it/s]


Validation Batch 2/4


Validation Loss: 7.256819725036621 Rouge1: 0.1702127659574468 Rouge2: 0.044444444444444446 RougeL: 0.12765957446808512: 100%|██████████| 5/5 [00:02<00:00,  2.40it/s]


Validation Batch 3/4


Validation Loss: 7.6357927322387695 Rouge1: 0.08333333333333334 Rouge2: 0.01694915254237288 RougeL: 0.05: 100%|██████████| 5/5 [00:01<00:00,  3.58it/s]


Validation Batch 4/4


Validation Loss: 7.5733113288879395 Rouge1: 0.11111111111111109 Rouge2: 0.0 RougeL: 0.07407407407407407: 100%|██████████| 5/5 [00:01<00:00,  3.86it/s]


In [ ]:
print(validation_rougeL_score)
generated

0.09566633242494751


['q : what is the caption for this figure? a : 9 the oft of number of a of with for of of number to number',
 'q : what is the caption for this figure? a : 14 work on task',
 'q : what is the caption for this figure? a : 1',
 'q : what is the caption for this figure? a : 3 diagram the of the of ofp',
 'q : what is the caption for this figure? a : 3 the architecture the of ofset',
 'q : what is the caption for this figure? a : 5 data of the of a class of',
 'q : what is the caption for this figure? a : 8',
 'q : what is the caption for this figure? a : ( 12 6',
 'q : what is the caption for this figure? a : 4 3 image the visual thea in a graph the of, a, d 10,, and a line the, in a, b a of and a to 10 2',
 'q : what is the caption for this figure? a : : 2 graphtion the of ins to and the of of of',
 'q : what is the caption for this figure? a : 10 the data for visual of in training',
 'q : what is the caption for this figure? a : 6 a diagram a',
 'q : what is the caption for this figure?

In [ ]:
LEARNING_RATE = 5e-5
SAVE_EVERY = 1000
TYPES = ["Unimodal", "Multimodal-Title", "Multimodal"]
TYPE = 0
SAVE_DIR = 'drive/MyDrive/Multimodal-Image-Captioning/Checkpoints/'+TYPES[TYPE]+'/'+DATASET+'/'
GRADIANT_ACCUMULATION_STEPS = 16
PER_BATCH = 40
VALIDATION_SIZE = 2
PER_VALIDATION_BATCH = 5

CURR_BATCH = 0
CHECKPOINT = None
#CHECKPOINT = 'drive/MyDrive/Multimodal-Image-Captioning/Checkpoints/'+TYPES[TYPE]+'/'+DATASET+'/'+str(CURR_BATCH)
#CHECKPOINT = 'drive/MyDrive/Multimodal-Image-Captioning/Checkpoints/'+TYPES[TYPE]+'/'+DATASET+'/model'

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").cuda()

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr=LEARNING_RATE)

In [ ]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(CHECKPOINT if CHECKPOINT is not None else "Salesforce/blip2-opt-2.7b").cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
for param in model.vision_model.parameters():
    param.requires_grad = False
for param in model.language_model.parameters():
    param.requires_grad = False

In [ ]:
backprops = 0
steps = 0
losses = []
validation_losses = []
validation_rougeL = []

for batch in range(CURR_BATCH, len(dataset)- 120):
    print(f'Batch {batch+1}')
    batch_data = dataset[batch]
    random.shuffle(batch_data)
    loop = tqdm(batch_data)
    for idx, data_point in enumerate(loop):
        try:
            loop.set_description(f'Running Model\tBackprops: {backprops} Steps: {steps} Loss: {losses[-1] if len(losses) > 0 else 0} Validation Loss: {validation_losses[-1] if len(validation_losses) > 0 else 0} Validation RougeL: {validation_rougeL[-1] if len(validation_rougeL) > 0 else 0}')
            out, loss = forward(model, processor, data_point, TYPE)
            losses.append(loss.item())
            loop.set_description(f'Backpropagating\tBackprops: {backprops} Steps: {steps} Loss: {losses[-1] if len(losses) > 0 else 0} Validation Loss: {validation_losses[-1] if len(validation_losses) > 0 else 0} Validation RougeL: {validation_rougeL[-1] if len(validation_rougeL) > 0 else 0}')
            loss.backward()
            backprops += 1
        except:
          print('Error')

        if backprops % GRADIANT_ACCUMULATION_STEPS == 0:
            loop.set_description(f'Applying Gradient\tBackprops: {backprops} Steps: {steps+1} Loss: {losses[-1] if len(losses) > 0 else 0} Validation Loss: {validation_losses[-1] if len(validation_losses) > 0 else 0} Validation RougeL: {validation_rougeL[-1] if len(validation_rougeL) > 0 else 0}')
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()
            optimizer.zero_grad()
            steps +=1



print("Saving Checkpoint")
model.save_pretrained(SAVE_DIR+"model_blip")

        #print("Validating")
        #validation_loss, validation_rouge1_score, validation_rouge2_score, validation_rougeL_score = validation(model, processor, dataset, scorer, TYPE, VALIDATION_SIZE, PER_VALIDATION_BATCH)
        #validation_losses.append(validation_loss)
        #validation_rouge1.append(validation_rouge1_score)
        #validation_rouge2.append(validation_rouge2_score)
        #validation_rougeL.append(validation_rougeL_score)
        #with open(SAVE_DIR+'/metrics.pkl', 'wb') as f:
        #    pickle.dump((losses, validation_losses, validation_rouge1, validation_rouge2, validation_rougeL), f)






Batch 1


Backpropagating	Backprops: 73 Steps: 4 Loss: 7.243788719177246 Validation Loss: 0 Validation RougeL: 0: 100%|██████████| 74/74 [00:11<00:00,  6.22it/s]


Batch 2


Running Model	Backprops: 79 Steps: 4 Loss: 5.572714328765869 Validation Loss: 0 Validation RougeL: 0:  12%|█▎        | 6/48 [00:00<00:04,  8.92it/s]  

Error


...

Applying Gradient	Backprops: 1024 Steps: 69 Loss: 5.366390705108643 Validation Loss: 0 Validation RougeL: 0: 100%|██████████| 43/43 [00:07<00:00,  6.14it/s]


Batch 20


Backpropagating	Backprops: 1088 Steps: 73 Loss: 4.034982204437256 Validation Loss: 0 Validation RougeL: 0: 100%|██████████| 65/65 [00:10<00:00,  6.07it/s]


Batch 21


KeyboardInterrupt: ignored